In [1]:
# 고차원의 저주
# 고차원이 되면 우리가 직관적으로 이해할 수 없는 특징이 생기고, 결과적으로 알고리즘이 잘 작동하지 않는다.
import numpy, scipy
d = 500
data = numpy.random.uniform(size=d*1000).reshape((1000,d))
distances = scipy.spatial.distance.cdist(data, data)
pd.Series(distances.reshape(1000000)).hist(bins=50)
plt.title("%i차원 공간에서 점 간 거리" %d)
plt.show()

AttributeError: module 'scipy' has no attribute 'spatial'

In [2]:
# 아이겐페이스와 차원 축소 예제
# 40명의 64x64 픽셀의 얼굴이미지 파일을 1명당 10장씩 불러옴. 64x64 = 4096차원 데이터
# 그 후 주성분 분석으로 4096차원의 데이터를 25차원으로 축소
import sklearn
import sklearn.datasets as datasets
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn import metrics

In [3]:
# 데이터 불러오기
faces_data = datasets.fetch_olivetti_faces()
person_ids, image_array = faces_data['target'], faces_data.images

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to C:\Users\jhpark\scikit_learn_data


In [7]:
# (64, 64) 크기의 데이터를 (4096, )으로 변환 (64 * 64 = 4096)
X = image_array.reshape((len(person_ids), 64*64))

In [8]:
# 군집화 알고리즘 실행
print("raw data clustering result")
model = KMeans(n_clusters=40)
model.fit(X)
print("clustering performance: ", silhouette_score(X, model.labels_))
print("face match rate: ", metrics.adjusted_rand_score(
        model.labels_,person_ids))

raw data clustering result
clustering performance:  0.14290293
face match rate:  0.41198686900146425


C:\Users\jhpark\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


In [9]:
# PCA
# pca.components_[0,:] 를 사용하면 첫번째 주성분을 볼 수 있음
# pd.Series(pca.explained_variance_ratio_).plot() 를 사용하면 몇번째까지 설명하는지 알 수 있음
# pca = PCA(n)
# pca.fit(X)
# X_reduced = pca.transform(X)
# 주성분 분석은 이상치에 민감. 모든 차원의 표준편차가 비슷해야한다.
print("After PCA, clustering result")
pca = PCA(25)
pca.fit(X)
X_reduced = pca.transform(X)
model_reduced = KMeans(n_clusters=40)
model_reduced.fit(X_reduced)
labels_reduced = model_reduced.labels_
print("clustering performance: ", silhouette_score(X_reduced, model_reduced.labels_))
print("face match rate: ", metrics.adjusted_rand_score(
        model_reduced.labels_,person_ids))

After PCA, clustering result
clustering performance:  0.23718557
face match rate:  0.43020323283385387


C:\Users\jhpark\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


In [1]:
# 군집화(clustering)
# 군집화는 연속적인 값을 이산적인 값으로 변환한다.
## 군집화 평가방법
# 각 군집마다 대푯값이 구별되어야 한다.
# 데이터가 고차원이라면 주성분 분석으로 2차원으로 축소한 뒤 산포도를 그림.
# 실루엣 점수를 통해 평가
# 정답을 알고 있다면 상호 정보량을 계싼해서 군집을 평가할수 있다.
## k-means clustering algorithm
# from sklearn.cluster import KMeans
# model = KMeans(n_clusters=k)
# model.fit(my_data)
# labels = model.labels_
# cluster_centers = model.cluster_centers_

# 상호정보량(mutual information)
# 두 확률변수 (x,y) 중 x의 값이 정해졌을 때 이 정보로 인해 주어지는 y의 정보량을 나타낸다.
# 예를 들어 두 변수가 독립이면 x와 y의 값에 아무런 연관이 없으므로 MI는 0이 나온다.
from sklearn import metrics
labels_true = [0,0,0,1,1,1]
labels_pred = [0,0,1,1,2,2]
metrics.mutual_info_score(labels_true, labels_pred)

0.4620981203732969